In [1]:
# Install Ultralytics YOLOv8
!pip install ultralytics
!pip install onnxruntime
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from ultralytics import YOLO
import cv2
from google.colab import files
import IPython.display as display
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Export to ONNX with opset=12 and dynamic axes
model.export(format='onnx', opset=12, dynamic=True)


100%|██████████| 6.25M/6.25M [00:00<00:00, 72.2MB/s]


Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 162.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 14.8s, installed 2 packages: ['onnx>=1.12.0', 'onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.50...
ONNX: export success ✅ 40.8s, saved as 'yolov8n.onnx' (12.1 MB)

Export complete (44.0s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Va

'yolov8n.onnx'

In [4]:
# Load YOLOv8 ONNX model
model = YOLO("yolov8n.onnx")
class_names = model.names

# Initialize DeepSORT tracker
deepsort = DeepSort(max_age=50, n_init=10, nn_budget=100)

# Upload a video file
uploaded = files.upload()  # Uncomment if running fresh
video_path = list(uploaded.keys())[0]

# Open video
cap = cv2.VideoCapture(video_path)

# Video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video writer
output_filename = "/content/deepsort_onnx.mp4"
out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Define areas (example: polygonal areas for entry/exit)
area1 =  np.array([(86, 230), (925, 230), (1000, 277), (3, 277)], np.int32)  # Entry area
area2 = np.array([(170, 180), (830, 180), (900, 215), (98, 215)], np.int32) # Exit area

# Counting logic
entered = set()
exited = set()
entering = {}
exiting = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference
    results = model.predict(source=frame, save=False, imgsz=640)

    # Prepare detections for DeepSORT
    detections = []
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            xyxy = box.xyxy[0].cpu().numpy().astype(int)

            if class_names[cls_id] == "person" and conf > 0.5:
                x1, y1, x2, y2 = xyxy
                w, h = x2 - x1, y2 - y1
                detections.append(([x1, y1, w, h], conf, "person"))  # Correct format

    # Update DeepSORT with formatted detections
    trackers = deepsort.update_tracks(detections, frame=frame)

    # Process trackers and check entry/exit
    for tracker in trackers:
        if not tracker.is_confirmed() or tracker.time_since_update > 0:
            continue

        bbox = tracker.to_tlbr()  # (x1, y1, x2, y2)
        track_id = tracker.track_id
        x1, y1, x2, y2 = map(int, bbox)

        # Draw bounding box and ID on frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Entering logic (Person enters area1)
        result2 = cv2.pointPolygonTest(np.array(area2, np.int32), (x2, y2), False)
        if result2 > -1 and track_id not in entering:
            entering[track_id] = (x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)

        if track_id in entering:
            result1 = cv2.pointPolygonTest(np.array(area1, np.int32), (x2, y2), False)
            if result1 > -1:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                entered.add(track_id)
                entering.pop(track_id)

        # Exiting logic (Person exits area2)
        result3 = cv2.pointPolygonTest(np.array(area1, np.int32), (x2, y2), False)
        if result3 > -1 and track_id not in exiting:
            exiting[track_id] = (x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)

        if track_id in exiting:
            result4 = cv2.pointPolygonTest(np.array(area2, np.int32), (x2, y2), False)
            if result4 > -1:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                exited.add(track_id)
                exiting.pop(track_id)

    # Draw areas
    cv2.polylines(frame, [np.array(area1, np.int32)], True, (255, 0, 0), 2)  # Entry area
    cv2.polylines(frame, [np.array(area2, np.int32)], True, (0, 255, 0), 2)  # Exit area

    # Display counts of entered and exited people
    cv2.putText(frame, f"Entered: {len(entered)}", (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(frame, f"Exited: {len(exited)}", (10, 60), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()

# Download output video
files.download(output_filename)


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8n.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider


Saving peoplecount.mp4 to peoplecount.mp4

0: 224x640 (no detections), 71.5ms
Speed: 8.8ms preprocess, 71.5ms inference, 7.5ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 (no detections), 65.9ms
Speed: 3.6ms preprocess, 65.9ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 (no detections), 66.7ms
Speed: 3.1ms preprocess, 66.7ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 1 person, 1 potted plant, 72.0ms
Speed: 4.0ms preprocess, 72.0ms inference, 21.3ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 1 person, 1 potted plant, 64.0ms
Speed: 3.3ms preprocess, 64.0ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 1 person, 66.8ms
Speed: 3.4ms preprocess, 66.8ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 640)

0: 224x640 1 person, 1 refrigerator, 63.3ms
Speed: 2.6ms preprocess, 63.3ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 640)

0: 22

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>